In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime

from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

In [2]:
x = rasterio_to_xarray('/Users/robin/ownCloud/HOTBAR/Annie/MAIACAAOT.h00v01.20140031240.hdf_proj.tif')

In [4]:
ds = x.to_dataset(name='data')

In [5]:
ds.to_netcdf('blah5.nc')

In [8]:
x.attrs

OrderedDict([('crs', '+init=epsg:27700'),
             ('affine',
              (-947639.6305106478,
               1256.5430440955893,
               0.0,
               1429277.8120091767,
               0.0,
               -1256.5430440955893))])

In [6]:
newds = xr.open_dataset('blah5.nc')

In [11]:
pwd

'/Users/robin/code/MAIACProcessing'

In [10]:
xarray_to_rasterio(newds['data'], 'attrstest.tif')

In [ ]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    da.values[da.values == -28672] = np.nan
    da.values[da.values == 0] = np.nan
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [ ]:
files = glob('2014/*_proj.tif')

In [ ]:
maiac_file_to_da(files[0])

In [ ]:
list_of_das = map(maiac_file_to_da, files)

In [ ]:
res = xr.concat(list_of_das, 'time')

In [ ]:
res

In [ ]:
r = res.groupby('time.date').max(dim='time')

In [ ]:
overall_mean = r.mean(dim='date', keep_attrs=True)

In [ ]:
m = r.mean(dim='date')

In [ ]:
xarray_to_rasterio(m, 'test_mnotattrs.tif')

In [ ]:
xarray_to_rasterio(overall_mean, 'test_overallmean.tif')

In [ ]:
import pandas as pd

In [ ]:
r['date'] = pd.to_datetime(r['date'])

In [ ]:
seasonal = r.groupby('date.month').mean(dim='date')

In [ ]:
[dim for dim in seasonal.dims if dim not in ['y', 'x']]

In [ ]:
seasonal.coords['month'].values

In [ ]:
xarray_to_rasterio(seasonal, 'test_seasonal.tif')

In [ ]:
figure(figsize=(20, 12))
seasonal.plot(col='month', robust=True)

In [ ]:
figure(figsize=(20, 12))
mr.plot.pcolormesh(robust=True)

In [ ]:
res.plot.pcolormesh??

In [ ]:
mr.sel(x=slice(0, 600000), y=slice(mr.y.min(), mr.y.max()))

In [ ]:
res.groupby('time.month').mean(dim='time')

In [ ]:
overall_count = r.count(dim='date')

In [ ]:
overall_count.isel(x=slice(1050,1150), y=slice(1000,1100)).plot(robust=True)

In [ ]:
overall_count.plot.hist()

In [ ]:
overall_mean.isel(x=slice(1050,1150), y=slice(1000,1100)).plot(robust=True)

In [ ]:
overall_mean.attrs

In [ ]:
r.coords['y'].max()

In [ ]:
r.sel(x=439480, y=138506, tolerance=1000, method='nearest').to_dataframe(name='data')